############# Readme ##############
The script comprises primarily of two functions the cut() function which returns the cut value for certain inputs and the QAOA function which produces a QAOA quantum circuit depending on the input graph and graph connections. The graph node number is encoded as a list of indexed qubits in V and the connections are encoded as tuples of node indexes in list E. The remainder of the scripy comprises of setting up the grid scan of the QAOA parameter space and performing tests of the algorithm on noiseless and noisy simulaotrs

In [171]:
from qiskit import *
import numpy as np
from qiskit.visualization import plot_histogram
import qiskit.tools.jupyter 
import matplotlib.pyplot as plt
import time

In [73]:
###### Encoding the Maxcut using Quantum

V=[0,1,2,3]  #Nodes of the graph
E=[[0,1],[1,2],[2,3],[3,0]] #Connections Of the graph




In [166]:
##Re-using bits from circuit simulator code
I=[[1,0],[0,1]]
Z=[[1,0],[0,-1]]
def sing_gate(K,l,N):       #Function creates matrix to represent single qubit gate K in index l in N qubit register
    master_list=[0]*N
    for i in range(len(master_list)):
        master_list[i]=I
    master_list[l]=K
    Matrix=master_list[0]
    for i in range(1,N):
        Matrix=np.kron(Matrix,master_list[i])
    return(Matrix)


In [167]:
#Computing the expectation value for cut operator using matrix representation.
#Used to find cut value of any input state.
def cut(inputs):
    inputs=list(inputs)
    for i in range(len(inputs)):
        inputs[i]=int(inputs[i]) #Converting bitstring inputs to string of boolean integers
    Cuts=[]
    for i in range (len(E)): #Computing the expectation value of a cut as outlined in reference
        
        N=len(inputs)
        a=E[i][0]
        b=E[i][1]
        Matrix=sing_gate(Z,a,len(V))*sing_gate(Z,b,len(V)) #Creating ZZ gate as outlined in reference
        Zero=[[1],[0]]
        One=[[0],[1]]
        if inputs[0]==0:                              #Creating input state from boolean integers of state that is to be tested
            input_state=Zero
            for i in range(1,N):
                if inputs[i]==0:
                    input_state=np.kron(input_state,Zero)
                if inputs[i]==1:
                    input_state=np.kron(input_state,One)
        if inputs[0]==1:
            input_state=One
            for i in range(1,N):
                if inputs[i]==0:
                    input_state=np.kron(input_state,Zero)
                if inputs[i]==1:
                    input_state=np.kron(input_state,One)
        expect=np.transpose(input_state)@Matrix@input_state
        
        Cuts.append(-expect)   #Making a list of cut values
    return(np.sum(Cuts))     #Summing list of cut values



In [199]:
def QAOA(a,b):     #a and b are parameters for separation and rotation matricies
    circuit=QuantumCircuit(len(V))  #Parameterizing length of circuit by number of nodes in graph
    for i in range(len(V)):
        circuit.h(i)                #Creating maximal superposition
    for i in range (len(E)):
        circuit.cx(E[i][0],E[i][1])   #Adding gates to impliment separation matrix
        circuit.p(a,E[i][1])
        circuit.cx(E[i][0],E[i][1])
    for i in range(len(V)):
        circuit.rx(2*b,i)            #Adding gates to impliment mixer matrix
    circuit.measure_all()
    return(circuit)
      

In [200]:
#######Implimenting QAOA for maxcut with graph and random connections
start_time=time.time()
V=[0,1,2,3,4,5,6]  #Nodes of the graph
E=[[0,1],[1,2],[1,2],[1,3],[0,3],[3,4],[1,4],[0,5],[3,5],[2,3],[3,0],[1,5],[0,4]] #Connections Of the graph
shots=10000
dict={}
a_values=np.linspace(0,2*np.pi,10)   #array of alpha values for classicalparameter grid scan
b_values=np.linspace(0,np.pi,10)#array of beta values for classicalparameter grid scan
for a in range(len(a_values)):
    for b in range(len(b_values)):
        counts = execute(QAOA(a_values[a],b_values[b]), sim, shots=shots).result().get_counts() #executing for certain a and b
        final_state=max(counts,key=counts.get)     #Getting most common state
        maxcut=cut(final_state)                   #Finding cut of most common state
        dict[(a_values[a], b_values[b],final_state)] = maxcut  #Placing a,b,output state, and maxcut produced into a dictionary
optimal_params = max(dict, key=lambda k: dict[k])  #Finding parameters which produced state with largest maxcut
maximum = dict[optimal_params]
end_time=time.time()
time_taken=end_time-start_time
print("Time taken for circuit with",len(V),"nodes:",time_taken)
print("Optimal Parameters and optimal state:", optimal_params)
print("Maximum Cut", maximum)

Time taken for circuit with 7 nodes: 8.216415643692017
Optimal Parameters and optimal state: (0.0, 2.443460952792061, '1010111')
Maximum Cut 7


In [ ]:
#######Finding the effect of varying beta values for mixer matrix-----Basically same as last cell with restricted grid search
start_time=time.time()
V=[0,1,2,3,4,5]  #Nodes of the graph
E=[[0,1],[1,2],[1,2],[1,3],[0,3],[3,4],[1,4]] #Connections Of the graph
shots=10000
dict={}
a=np.pi/4 #Keeping a value constant
b_values=np.linspace(0,np.pi,10)
for b in range(len(b_values)):
        counts = execute(QAOA(a,b_values[b]), sim, shots=shots).result().get_counts()
        final_state=max(counts,key=counts.get)
        maxcut=cut(final_state)
        dict[(a, b_values[b],final_state)] = maxcut
        #optimal_params = max(dict, key=lambda k: dict[k])
        #maximum = dict[optimal_params]
end_time=time.time()
time_taken=end_time-start_time
#print("Time taken for circuit with",len(V),"nodes:",time_taken)
#print("Optimal Parameters and optimal state:", optimal_params)
#print("Maximum Cut", maximum)
print(dict)

In [191]:
from qiskit.providers.fake_provider import FakeMelbourne
backend = FakeMelbourne()
shots=10000

In [196]:
#######Implimenting QAOA for maxcut on noisy backend--Same as cell with noiseless simulation but replaced with optimized noist backend
start_time=time.time()
V=[0,1,2,3,4,5,6]  #Nodes of the graph
E=[[0,1],[1,2],[1,2],[1,3],[0,3],[3,4],[1,4],[0,5],[3,5],[2,3],[3,0],[1,5],[0,4]] #Connections Of the graph
dict={}
a_values=np.linspace(0,2*np.pi,10)
b_values=np.linspace(0,np.pi,10)
for a in range(len(a_values)):
    for b in range(len(b_values)):
        noisy_simulation = transpile(QAOA(a_values[a],b_values[b]), backend, optimization_level=3)     #Optimising QAOA circuit
        counts = execute(QAOA(a_values[a],b_values[b]), backend, shots=shots).result().get_counts()    #Executing on noisy simulator
        final_state=max(counts,key=counts.get)
        maxcut=cut(final_state)
        dict[(a_values[a], b_values[b],final_state)] = maxcut
        optimal_params = max(dict, key=lambda k: dict[k])
        maximum = dict[optimal_params]
end_time=time.time()
time_taken=end_time-start_time
print("Time taken for circuit with",len(V),"nodes:",time_taken)
print("Optimal Parameters and optimal state:", optimal_params)
print("Maximum Cut", maximum)

Time taken for circuit with 7 nodes: 69.9832112789154
Optimal Parameters and optimal state: (1.3962634015954636, 0.6981317007977318, '1001001')
Maximum Cut 1
